In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [3]:
dataset = load_boston()
dataset.data.shape

(506, 13)

In [5]:
x_full,y_full = dataset.data,dataset.target
n_sample = x_full.shape[0]
n_feature = x_full.shape[1]
x_full.shape

(506, 13)

## 为完整数据放入缺失值
确认我们希望放入缺失值等比例，在这里我们假设是50%，那总共就要有3289个数据缺失

In [7]:
rng = np.random.RandomState(0)
missing_rate = 0.5
n_missing_samples = int(np.floor(n_sample*n_feature*missing_rate))
n_missing_samples

3289

In [30]:
# n_feature = 13 , n_sample = 506
missing_feature = rng.randint(0,n_feature,n_missing_samples)
missing_sample = rng.randint(0,n_sample,n_missing_samples)
# choice
# missing_sample = rng.choice(n_sample,n_missing_samples,replace=false)


In [36]:
x_missing = x_full.copy()
y_missing = y_full.copy()
# 测试语法
# x_missing[[0,1],[1,2]] = 22
# x_missing[[[0,1,2]],[[1,2,3]]] = 22
# x_missing

In [37]:
x_missing[missing_sample,missing_feature] = np.nan
# x_missing

In [38]:
x_missing = pd.DataFrame(x_missing)
x_missing

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,NaN,NaN,0.538,NaN,65.2,4.0900,1.0,296.0,15.3,NaN,4.98
1,NaN,NaN,7.07,0.0,0.469,6.421,NaN,4.9671,NaN,242.0,17.8,396.90,NaN
2,0.02729,0.0,7.07,0.0,NaN,NaN,61.1,4.9671,2.0,NaN,NaN,392.83,NaN
3,NaN,0.0,NaN,NaN,0.458,NaN,NaN,NaN,NaN,222.0,NaN,NaN,NaN
4,0.06905,NaN,2.18,0.0,0.458,NaN,54.2,6.0622,3.0,NaN,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,NaN,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,NaN,0.0,11.93,NaN,0.573,6.120,NaN,2.2875,NaN,273.0,21.0,396.90,NaN
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,NaN,NaN,NaN,21.0,NaN,NaN
504,0.10959,NaN,11.93,0.0,0.573,6.794,89.3,NaN,1.0,NaN,NaN,393.45,NaN


In [43]:
# 使用 平均数 或者中位数 填充Age
simple_inputer = SimpleImputer();#默认均值
median_inputer = SimpleImputer(strategy="median");#中位数
zero_inputer = SimpleImputer(strategy="constant",fill_value=0);#用0填补


# x_missing_mean = simple_inputer.fit_transform(x_missing)
# x_missing_mean

x_missing_zero = zero_inputer.fit_transform(x_missing)
x_missing_zero

array([[6.3200e-03, 1.8000e+01, 0.0000e+00, ..., 1.5300e+01, 0.0000e+00,
        4.9800e+00],
       [0.0000e+00, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
        0.0000e+00],
       [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 0.0000e+00, 3.9283e+02,
        0.0000e+00],
       ...,
       [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 0.0000e+00,
        0.0000e+00],
       [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 0.0000e+00, 3.9345e+02,
        0.0000e+00],
       [4.7410e-02, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 3.9690e+02,
        7.8800e+00]])

In [46]:
# 检查缺失值
x_missing_zero = pd.DataFrame(x_missing_zero)
x_missing_zero.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
dtype: int64